<a href="https://colab.research.google.com/github/yejijang-analyst/ESAA/blob/main/exercise/24-1/week2_Traininglinearmodels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **| 모델 훈련 연습 문제**
___
- 출처 : 핸즈온 머신러닝 Ch04 연습문제 1, 5, 9, 10
- 개념 문제의 경우 텍스트 셀을 추가하여 정답을 적어주세요.

### **1. 수백만 개의 특성을 가진 훈련 세트에서는 어떤 선형 회귀 알고리즘을 사용할 수 있을까요?**
___


미니배치 , SGD, 배치 GD 를 활용해볼 수 있다.

### **2. 배치 경사 하강법을 사용하고 에포크마다 검증 오차를 그래프로 나타내봤습니다. 검증 오차가 일정하게 상승되고 있다면 어떤 일이 일어나고 있는 걸까요? 이 문제를 어떻게 해결할 수 있나요?**
___

lr 이 너무 크게되어있을 확률이 높으니 낮게 조절해보는 것이 필요할듯. 훈련에러도 올라가는지 확인하여 맞으면 lr 조정.

### **3. 릿지 회귀를 사용했을 때 훈련 오차가 검증 오차가 거의 비슷하고 둘 다 높았습니다. 이 모델에는 높은 편향이 문제인가요, 아니면 높은 분산이 문제인가요? 규제 하이퍼파라미터 $\alpha$를 증가시켜야 할까요 아니면 줄여야 할까요?**
___

훈련 오차와 검증 오차가 거의 비슷하고 매우 높다면 모델이 훈련 세트에 과소적합되었을 가능성이 높다. 즉, 높은 편향을 가진 모델이다. 따라서 규제 하이퍼파라미터 α를 감소시켜야한다. α를 증가시킬수록 복잡도가 떨어짐. α를 낮춰야 분산이 증가하고 편향이 감소함.


### **4. 다음과 같이 사용해야 하는 이유는?**
___
- 평범한 선형 회귀(즉, 아무런 규제가 없는 모델) 대신 릿지 회귀
- 릿지 회귀 대신 라쏘 회귀
- 라쏘 회귀 대신 엘라스틱넷

평범한 선형 회귀 (아무런 규제가 없음) 대신 릿지 회귀: 규제가 있는 모델이 규제가 없는 모델에 비해 성능이 좋다. 그래서 평범한 회귀보다는 릿지 회귀가 선호된다.

릿지 회귀 대신 라쏘 회귀: 라쏘 회귀는 l1 패널티를 사용하여 가중치를 완전히 0으로 만드는 경향이 있다. 가장 중요한 가중치를 제외하고는 모두 0이 되는 희소한(Sparse)한 모델을 만든다. 그리고 "자동으로 특성 선택"의 효과를 가지므로 단지 몇 개의 특성만 실제 유용할 것이라고 의심될 때 사용하면 좋다. 확신이 없다면 릿지 회귀를 사용해야 합니다.

라쏘회귀 대신 엘라스틱넷: 몇 개의 특성이 강하게 연관되어 있거나 / 특성 수가 훈련 샘플보다 많을 때 라쏘가 불규칙적으로 행동하여 엘라스틱넷이 더 선호된다. 하지만 추가적인 하이퍼파라미터가 생기게 된다. 불규칙한 행동이 없는 라쏘를 원하면 엘라스틱넷에 l1_ratio를 1에 가깝게 하면 된다. (ratio = 0(Ridge) ~ 1(Lasso))


### **추가) 조기 종료를 사용한 배치 경사 하강법으로 iris 데이터를 활용해 소프트맥스 회귀를 구현해보세요(사이킷런은 사용하지 마세요)**


---



In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from sklearn.datasets import load_iris

iris = load_iris()
list(iris.keys())

In [ ]:

X = iris.data[:, (2,3)] # 꽃잎 길이, 꽃잎 너비
y = iris["target"]


In [ ]:

import numpy as np
X_b = np.c_[np.ones(len(X)), X]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_b, y, test_size = 0.2, random_state=42)

In [ ]:

from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(categories='auto')
y_train_ohe = encoder.fit_transform(y_train.reshape(-1, 1)).toarray()
y_test_ohe = encoder.fit_transform(y_test.reshape(-1, 1)).toarray()


In [ ]:
def softmax(z):
  exps = np.exp(z)
  exp_sum = np.sum(exps, axis=1, keepdims = True)
  return exps/exp_sum

In [ ]:
eta = 0.1 # 학습률
n_epochs = 1000
m = len(X_train) # 샘플 개수
history=[]
epsilon = 1e-7
np.random.seed(42)


for epoch in range(n_epochs):
  Y_proba = softmax(s_x)
  loss = -np.mean(np.sum(y_train_ohe * np.log(Y_proba + epsilon), axis=1))
  error = Y_proba - y_train_ohe # (#,3)
  gradients = (1/m) * (X_train.T.dot(error))
  theta = theta - eta * gradients
  history.append(loss)